xxx yolos

what do we need for yolo?
get all the labeld images?

transfrome labels?

weights?


In [ ]:
from transformers import AutoModelForObjectDetection,AutoImageProcessor
from PIL import Image, ImageDraw
from functools import partial

import pandas as pd
import torch
import cv2 as cv
import numpy as np
import shutil
import os
import json


In [ ]:
image_processor = AutoImageProcessor.from_pretrained("hustvl/yolos-base")
model = AutoModelForObjectDetection.from_pretrained("hustvl/yolos-base")

In [ ]:
img = Image.open("tomo_00e463/slice_0235.jpg")
test_img=image_processor(images=img, return_tensors="pt")


In [ ]:
img

In [ ]:

outputs = model(**test_img)
#model.bbox_predictor(**test_img)

In [ ]:
target_sizes = torch.tensor([img.size[::-1]])

results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[

    0

]

In [ ]:
from transformers import YolosFeatureExtractor, YolosForObjectDetection

grayscale_image = Image.open("tomo_00e463/slice_0235.jpg").convert("L")

# Convert to RGB
rgb_image = Image.merge("RGB", (grayscale_image, grayscale_image, grayscale_image))

# Save the RGB image
rgb_image.save("rgb_test_img.jpg")
img = Image.open("rgb_test_img.jpg")

feature_extractor = YolosFeatureExtractor.from_pretrained('hustvl/yolos-base')
model = YolosForObjectDetection.from_pretrained('hustvl/yolos-base')

inputs = feature_extractor(images=img, return_tensors="pt")
outputs = model(**inputs)

In [ ]:
logits = outputs.logits
bboxes = outputs.pred_boxes

In [ ]:
bboxes

In [ ]:
target_sizes = torch.tensor([[img.size[1], img.size[0]]])
results = image_processor.post_process_object_detection(outputs, threshold=0.005, target_sizes=target_sizes)[0]


In [ ]:
results

In [ ]:
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):

    box = [round(i, 2) for i in box.tolist()]

    print(

        f"Detected {model.config.id2label[label.item()]} with confidence "

        f"{round(score.item(), 3)} at location {box}"

    )

In [ ]:
draw = ImageDraw.Draw(img)

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):

    box = [round(i, 2) for i in box.tolist()]

    x, y, x2, y2 = tuple(box)

    draw.rectangle((x, y, x2, y2), outline="red", width=1)

    draw.text((x, y), model.config.id2label[label.item()], fill="white")

img

---
## Fine Tune/ Transfer Modell

In [ ]:
df=pd.read_csv("train_labels.csv")

In [ ]:
df.head(2)

In [ ]:
df["Voxel spacing"].value_counts()

In [ ]:
print(df[df["Motor axis 0"]!=-1]["Motor axis 0"].nsmallest(5))
print(df[df["Motor axis 1"]!=-1]["Motor axis 1"].nsmallest(5))
print(df[df["Motor axis 2"]!=-1]["Motor axis 2"].nsmallest(5))

In [ ]:
print(df["Array shape (axis 0)"].unique())
print(df["Array shape (axis 1)"].unique())
print(df["Array shape (axis 2)"].unique())

In [ ]:
#print(len(df[df["Array shape (axis 1)"]!=959]))
print(len(df[df["Array shape (axis 1)"]!=959]))
print(len(df[df["Array shape (axis 2)"]!=928]))

In [ ]:
print(len(df[df["Motor axis 0"]!=-1]))
print(len(df[df["Motor axis 1"]!=-1]))
print(len(df[df["Motor axis 2"]!=-1]))

In [ ]:
print(len(df[df["Motor axis 0"]!=-1]["tomo_id"].unique()))
print(len(df[df["Motor axis 1"]!=-1]["tomo_id"].unique()))
print(len(df[df["Motor axis 2"]!=-1]["tomo_id"].unique()))

In [ ]:
df[df["Motor axis 0"]==466]

In [ ]:
df[df["Motor axis 2"]!=-1]["Motor axis 2"].min()

In [ ]:
df_bbox=df.iloc[:,1:6]
labels=df_bbox.values
labels

In [ ]:
# Create YOLO format label
# YOLO format: <class> <x_center> <y_center> <width> <height>
# Values are normalized to [0, 1]
# {"file_name": "0001.png", "objects": {"bbox": [[302.0, 109.0, 73.0, 52.0]], "categories": [0]}}
BBOXS_SIZE = 70
detections = []
detections_empty = []
trust = 4
rng = np.random.default_rng(42)
meta_train = []
meta_test = []
size_empty = 3

for a in labels:
    if a[1] != -1:
        for b in np.arange(int(a[1])-trust, int(a[1])+trust+1):
            if b < 0:
                continue
            width = BBOXS_SIZE
            height = BBOXS_SIZE
            if BBOXS_SIZE/2 > a[-2]:
                width = (a[-2])*2
            if BBOXS_SIZE/2 > a[2]:
                height = (a[2]-1)*2
            detections.append({"file_name": f"{a[0]}/slice_{b:04d}.jpg", "objects": {
                              "bbox": [[a[-2], a[2], width, height]], "categories": [0], "id" : [],"area":[]}})
    else:
        random_slice = rng.integers(low=0, high=a[-1], size=size_empty)

        for c in random_slice:
            detections_empty.append(
                {"file_name": f"{a[0]}/slice_{c:04d}.jpg", "objects": {"bbox": [[]], "categories": []}})

print(len(detections_empty+detections))

print(f"portion of empty = {len(detections_empty)/len(detections)}")

# collect train metadata.jason
for i in detections:
    if i["objects"]["bbox"][0][1] >= 0:
        source = "train/"+i["file_name"]
        destination = "data/train/"+i["file_name"]

        if not os.path.isdir(os.path.dirname(destination)):
            os.mkdir(os.path.dirname(destination))

        shutil.copyfile(source, destination)
        meta_train.append(i)

for i in detections_empty:

    source = "train/"+i["file_name"]
    destination = "data/train/"+i["file_name"]

    if not os.path.isdir(os.path.dirname(destination)):
        os.mkdir(os.path.dirname(destination))

    shutil.copyfile(source, destination)
    meta_train.append(i)


# Train test split
split = 0.1
size_det = len(detections)
size_empt = len(detections_empty)

test_split_det = int(size_det*split)
test_split_empt = int(size_empt*split)

indices_test_det = rng.integers(low=0, high=size_det, size=test_split_det)
indices_test_empt = rng.integers(low=0, high=size_empt, size=test_split_empt)


# copy test files


for i in indices_test_det:
    if detections[i]["objects"]["bbox"][0][1] >= 0:
        source = "train/"+detections[i]["file_name"]
        destination = "data/test/"+detections[i]["file_name"]

        if not os.path.isdir(os.path.dirname(destination)):
            os.mkdir(os.path.dirname(destination))

        shutil.copyfile(source, destination)
        meta_test.append(detections[i])


for i in indices_test_empt:

    source = "train/"+detections_empty[i]["file_name"]
    destination = "data/test/"+detections_empty[i]["file_name"]

    if not os.path.isdir(os.path.dirname(destination)):
        os.mkdir(os.path.dirname(destination))

    shutil.copyfile(source, destination)
    meta_test.append(detections_empty[i])


# write to jason for hugginface



with open("data/train/metadata.jsonl", 'w') as jsonl_output:
    for entry in meta_train:
        json.dump(entry, jsonl_output)
        jsonl_output.write('\n')
        

with open("data/test/metadata.jsonl", 'w') as jsonl_output:
    for entry in meta_test:
        json.dump(entry, jsonl_output)
        jsonl_output.write('\n')

"""
with open("data/train/metadata.json", "w") as outfile:
    json.dump(meta_train, outfile)


with open("data/test/metadata.json", "w") as outfile:
    json.dump(meta_test, outfile)"""


# Create YAML configuration file for YOLO
"""yaml_content = {
    'path': yolo_dataset_dir,
    'train': 'images/train',
    'val': 'images/val',
    'names': {0: 'motor'}
}"""
print(len(indices_test_det)+len(indices_test_empt))


In [ ]:
from datasets import load_dataset
dataset=[]
# Load dataset
dataset = load_dataset("imagefolder", data_dir="data/")

#id2label = {0: "motor"}

In [ ]:
if "validation" not in dataset:

    split = dataset["train"].train_test_split(0.075, seed=1337)

    dataset["train"] = split["train"]

    dataset["validation"] = split["test"]

In [ ]:
dataset

In [ ]:
dataset["train"]["image"][0]

In [ ]:
dataset["train"][1]

In [ ]:
image = dataset["train"][2]["image"]

annotations = dataset["train"][2]["objects"]

annotations["bbox"][0]

In [ ]:
image = dataset["train"][2]["image"]

annotations = dataset["train"][2]["objects"]


draw = ImageDraw.Draw(image)

box = annotations["bbox"][0]

x, y, w, h = tuple(box)


draw.rectangle((x-w/2, y-h/2, x + w/2, y + h/2), outline="white", width=1)

draw.text((x-w/2, y-h/2), "Motor", fill="black")

image

In [ ]:
MAX_SIZE= 924 

image_processor = AutoImageProcessor.from_pretrained(

    "hustvl/yolos-base",

    do_resize=True,

    size={"max_height": MAX_SIZE, "max_width": MAX_SIZE},

    do_pad=True,

    pad_size={"height": MAX_SIZE, "width": MAX_SIZE},

)

In [ ]:
import albumentations as A

train_augment_and_transform = A.Compose(

    [

        A.Perspective(p=0.1),

        A.HorizontalFlip(p=0.5),

        A.RandomBrightnessContrast(p=0.5),

        A.HueSaturationValue(p=0.1),

    ],

    bbox_params=A.BboxParams(format="coco", clip=True, min_area=25),

)

validation_transform = A.Compose(

    [A.NoOp()],

    bbox_params=A.BboxParams(format="coco", clip=True),

)


In [ ]:

def format_image_annotations_as_coco(image_id, bboxes):

    """Format one set of image annotations to the COCO format

    Args:

        image_id (str): image id. e.g. "0001"

        categories (List[int]): list of categories/class labels corresponding to provided bounding boxes

        areas (List[float]): list of corresponding areas to provided bounding boxes

        bboxes (List[Tuple[float]]): list of bounding boxes provided in COCO format

            ([center_x, center_y, width, height] in absolute coordinates)

    Returns:

        dict: {

            "image_id": image id,

            "annotations": list of formatted annotations

        }

    """

    annotations = []

    for bbox in bboxes:

        formatted_annotation = {

            "image_id": image_id,

            "category_id": 0,

            "iscrowd": 0,

            "area": 0

            "bbox": list(bbox),

        }

        annotations.append(formatted_annotation)

    return {

        "image_id": image_id,

        "annotations": annotations,

    }

In [ ]:
dataset["train"][2]["image"]

In [ ]:
train_augment_and_transform(dataset["train"][2]["image"],dataset["train"][2]["objects"])

In [ ]:
def augment_and_transform_batch(examples, transform, image_processor, return_pixel_mask=False):

    """Apply augmentations and format annotations in COCO format for object detection task"""

    images = []

    annotations = []

    for image, objects in zip(examples["image"], examples["objects"]):

        image = np.array(image.convert("RGB"))

        # apply augmentations

        output = transform(image=image, bboxes=objects["bbox"], category=objects["category"])

        images.append(output["image"])

        # format annotations in COCO format

        formatted_annotations = format_image_annotations_as_coco(

            output["category"], objects["area"], output["bboxes"]

        )

        annotations.append(formatted_annotations)

    # Apply the image processor transformations: resizing, rescaling, normalization

    result = image_processor(images=images, annotations=annotations, return_tensors="pt")

    if not return_pixel_mask:

        result.pop("pixel_mask", None)

    return result

In [ ]:
train_transform_batch = partial(

    augment_and_transform_batch, transform=train_augment_and_transform, image_processor=image_processor

)

validation_transform_batch = partial(

    augment_and_transform_batch, transform=validation_transform, image_processor=image_processor

)

dataset["train"] = dataset["train"].with_transform(train_transform_batch)

dataset["validation"] = dataset["validation"].with_transform(validation_transform_batch)

dataset["test"] = dataset["test"].with_transform(validation_transform_batch)

dataset["train"][15]

In [ ]:
def convert_bbox_yolo_to_pascal(boxes, image_size):

    """

    Convert bounding boxes from YOLO format (x_center, y_center, width, height) in range [0, 1]

    to Pascal VOC format (x_min, y_min, x_max, y_max) in absolute coordinates.

    Args:

        boxes (torch.Tensor): Bounding boxes in YOLO format

        image_size (Tuple[int, int]): Image size in format (height, width)

    Returns:

        torch.Tensor: Bounding boxes in Pascal VOC format (x_min, y_min, x_max, y_max)

    """

    # convert center to corners format

    boxes = center_to_corners_format(boxes)

    # convert to absolute coordinates

    height, width = image_size

    boxes = boxes * torch.tensor([[width, height, width, height]])

    return boxes

In [ ]:
from dataclasses import dataclass

from torchmetrics.detection.mean_ap import MeanAveragePrecision
id2label = {0: "Motor"}
label2id = {"motor":0}
@dataclass

class ModelOutput:

    logits: torch.Tensor

    pred_boxes: torch.Tensor


@torch.no_grad()

def compute_metrics(evaluation_results, image_processor, threshold=0.0, id2label=None):

    """

    Compute mean average mAP, mAR and their variants for the object detection task.

    Args:

        evaluation_results (EvalPrediction): Predictions and targets from evaluation.

        threshold (float, optional): Threshold to filter predicted boxes by confidence. Defaults to 0.0.

        id2label (Optional[dict], optional): Mapping from class id to class name. Defaults to None.

    Returns:

        Mapping[str, float]: Metrics in a form of dictionary {<metric_name>: <metric_value>}

    """

    predictions, targets = evaluation_results.predictions, evaluation_results.label_ids

    # For metric computation we need to provide:

    #  - targets in a form of list of dictionaries with keys "boxes", "labels"

    #  - predictions in a form of list of dictionaries with keys "boxes", "scores", "labels"

    image_sizes = []

    post_processed_targets = []

    post_processed_predictions = []

    # Collect targets in the required format for metric computation

    for batch in targets:

        # collect image sizes, we will need them for predictions post processing

        batch_image_sizes = torch.tensor(np.array([x["orig_size"] for x in batch]))

        image_sizes.append(batch_image_sizes)

        # collect targets in the required format for metric computation

        # boxes were converted to YOLO format needed for model training

        # here we will convert them to Pascal VOC format (x_min, y_min, x_max, y_max)

        for image_target in batch:

            boxes = torch.tensor(image_target["boxes"])

            boxes = convert_bbox_yolo_to_pascal(boxes, image_target["orig_size"])

            labels = torch.tensor(image_target["class_labels"])

            post_processed_targets.append({"boxes": boxes, "labels": labels})

    # Collect predictions in the required format for metric computation,

    # model produce boxes in YOLO format, then image_processor convert them to Pascal VOC format

    for batch, target_sizes in zip(predictions, image_sizes):

        batch_logits, batch_boxes = batch[1], batch[2]

        output = ModelOutput(logits=torch.tensor(batch_logits), pred_boxes=torch.tensor(batch_boxes))

        post_processed_output = image_processor.post_process_object_detection(

            output, threshold=threshold, target_sizes=target_sizes

        )

        post_processed_predictions.extend(post_processed_output)

    # Compute metrics

    metric = MeanAveragePrecision(box_format="xyxy", class_metrics=True)

    metric.update(post_processed_predictions, post_processed_targets)

    metrics = metric.compute()

    # Replace list of per class metrics with separate metric for each class

    classes = metrics.pop("classes")

    map_per_class = metrics.pop("map_per_class")

    mar_100_per_class = metrics.pop("mar_100_per_class")

    for class_id, class_map, class_mar in zip(classes, map_per_class, mar_100_per_class):

        class_name = id2label[class_id.item()] if id2label is not None else class_id.item()

        metrics[f"map_{class_name}"] = class_map

        metrics[f"mar_100_{class_name}"] = class_mar

    metrics = {k: round(v.item(), 4) for k, v in metrics.items()}

    return metrics


eval_compute_metrics_fn = partial(

    compute_metrics, image_processor=image_processor, id2label=id2label, threshold=0.0

)

In [ ]:
def collate_fn(batch):

    data = {}

    data["pixel_values"] = torch.stack([x["pixel_values"] for x in batch])

    data["labels"] = [x["labels"] for x in batch]

    if "pixel_mask" in batch[0]:

        data["pixel_mask"] = torch.stack([x["pixel_mask"] for x in batch])

    return data

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(

    output_dir="detr_finetuned_cppe5",

    num_train_epochs=30,

    fp16=False,

    per_device_train_batch_size=8,

    dataloader_num_workers=4,

    learning_rate=5e-5,

    lr_scheduler_type="cosine",

    weight_decay=1e-4,

    max_grad_norm=0.01,

    metric_for_best_model="eval_map",

    greater_is_better=True,

    load_best_model_at_end=True,

    eval_strategy="epoch",

    save_strategy="epoch",

    save_total_limit=2,

    remove_unused_columns=False,

    eval_do_concat_batches=False,

    push_to_hub=False,

)

In [ ]:
from transformers import AutoModelForObjectDetection

model = AutoModelForObjectDetection.from_pretrained(

    "hustvl/yolos-base",

    id2label=id2label,

    label2id=label2id,

    ignore_mismatched_sizes=True,

)

In [ ]:
from transformers import Trainer

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=dataset["train"],

    eval_dataset=dataset["validation"],

    processing_class=image_processor,

    data_collator=collate_fn,

    compute_metrics=eval_compute_metrics_fn,

)

trainer.train()